In [13]:
import sqlite3

conexion = sqlite3.connect("database.db")
cursor = conexion.cursor()

In [9]:
cursor.execute('DROP TABLE IF EXISTS usuarios')

ProgrammingError: Cannot operate on a closed database.

In [5]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS usuarios (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    negocio_id INTEGER NOT NULL,
    nombre TEXT NOT NULL,
    dni TEXT NOT NULL UNIQUE,
    username TEXT NOT NULL UNIQUE,
    email TEXT,
    password TEXT NOT NULL,
    rol INTEGER NOT NULL,
    PHONE TEXT NOT NULL,
    foto TEXT,
    fecha_creacion TEXT NOT NULL,
    FOREIGN KEY (negocio_id) REFERENCES negocios (negocio_id)
)
''')

In [ ]:

from datetime import datetime

fecha_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Insertar negocio de prueba si no existe
cursor.execute('''
INSERT INTO negocios (nombre, sector, pais, region, ciudad, direccion, telefono, RUC, foto, descripcion, tipo_suscripcion, fecha_creacion)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', ('Negocio Ejemplo', 'Retail', 'Perú', 'Lima', 'Lima', 'Av. Siempre Viva 123', '123456789', '12345678901', '', 'Negocio de prueba', 'Gratis', fecha_actual))

# Obtener el ID del negocio recién insertado
negocio_id = cursor.lastrowid

# Insertar usuarios con roles del 1 al 4
usuarios = [
    ('Juan Pérez', '12345678', 'juan1@example.com', 'pass1', '1', '900000001'),
    ('Ana López', '87654321', 'ana2@example.com', 'pass2', '2', '900000002'),
    ('Luis Torres', '11223344', 'luis3@example.com', 'pass3', '3', '900000003'),
    ('Carmen Díaz', '44332211', 'carmen4@example.com', 'pass4', '4', '900000004')
]

for nombre, dni, email, password, rol, phone in usuarios:
    cursor.execute('''
    INSERT INTO usuarios (negocio_id, nombre, dni, username, email, password, rol, PHONE, fecha_creacion)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (negocio_id, nombre, dni, dni, email, password, rol, phone, fecha_actual))
conexion.commit()
conexion.close()

In [15]:
# nombre_del_script.py
import os
import sys
import django
from getpass import getpass # Para ocultar la contraseña al escribirla

def setup_django_environment():
    """Configura el entorno para poder usar los modelos de Django."""
    # --- ¡CONFIGURA ESTAS RUTAS! ---
    # Ruta a la carpeta PRINCIPAL de tu proyecto (la que contiene manage.py)
    project_path = r'C:/Users/Rudy/Documents/Proyectos/SVIC'  # <--- CAMBIA ESTO
    # Nombre de la carpeta que contiene tu settings.py
    project_settings_folder = 'SVIC' # <--- CAMBIA ESTO si tu carpeta de settings se llama diferente
    # --- FIN DE LA CONFIGURACIÓN ---

    if not os.path.isdir(project_path):
        print(f"Error: La ruta del proyecto '{project_path}' no existe o no es un directorio.")
        sys.exit(1)

    # Añadir la ruta del proyecto al sys.path para que Python encuentre tus apps
    sys.path.append(project_path)

    # Establecer la variable de entorno para que Django sepa qué settings usar
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_settings_folder}.settings')

    try:
        # Inicializar Django
        django.setup()
        print("Entorno Django configurado exitosamente.")
    except Exception as e:
        print(f"Error configurando Django: {e}")
        print("Asegúrate de que la variable 'project_path' y 'project_settings_folder' sean correctas.")
        print(f"DJANGO_SETTINGS_MODULE se intentó establecer como: {project_settings_folder}.settings")
        sys.exit(1)

def add_user_interactively():
    """Función principal para añadir un usuario pidiendo datos."""
    # Importar modelos DESPUÉS de django.setup()
    try:
        from usuarios.models import Usuario, Negocio
        from django.db import IntegrityError
        from django.core.exceptions import ObjectDoesNotExist
    except ImportError as e:
        print(f"Error importando modelos: {e}")
        print("Asegúrate de que django.setup() se ejecutó correctamente y que las apps están en INSTALLED_APPS.")
        sys.exit(1)

    print("\n--- Agregar Nuevo Usuario ---")

    # --- 1. Obtener y validar el Negocio ---
    negocio_instance = None
    while negocio_instance is None:
        try:
            print("\nNegocios disponibles:")
            negocios_disponibles = list(Negocio.objects.all())
            if not negocios_disponibles:
                print("Error: No hay negocios registrados en la base de datos.")
                print("Debes crear al menos un negocio antes de añadir usuarios.")
                return # Salir de la función si no hay negocios
            for negocio in negocios_disponibles:
                print(f"- ID: {negocio.id}, Nombre: {negocio.nombre}")

            negocio_id_str = input("Introduce el ID del Negocio al que pertenece el usuario: ")
            negocio_id = int(negocio_id_str)
            negocio_instance = Negocio.objects.get(pk=negocio_id)
            print(f"-> Seleccionado Negocio: {negocio_instance.nombre} (ID: {negocio_instance.id})")
        except ValueError:
            print("Error: El ID debe ser un número entero.")
        except ObjectDoesNotExist:
            print(f"Error: No existe un Negocio con ID '{negocio_id_str}'. Intenta de nuevo.")
        except Exception as e:
            print(f"Ocurrió un error inesperado buscando el negocio: {e}")
            return # Salir en caso de error grave

    # --- 2. Obtener datos del Usuario ---
    print("\nIntroduce los datos del nuevo usuario:")
    dni = input("   DNI: ").strip()
    nombre = input("   Nombre completo: ").strip()
    email = input("   Email (opcional, presiona Enter si no aplica): ").strip()
    phone = input("   Teléfono: ").strip()

    rol = None
    while rol is None:
        try:
            rol_str = input("   Rol (1: CEO, 2: Admin, 3: Inventario, 4: Ventas): ").strip()
            rol_int = int(rol_str)
            if rol_int in [1, 2, 3, 4]:
                rol = rol_int
            else:
                print("   Error: Rol inválido. Debe ser 1, 2, 3 o 4.")
        except ValueError:
            print("   Error: El rol debe ser un número entero.")

    # --- 3. Obtener y confirmar contraseña ---
    password = ""
    while not password:
        password = getpass("   Contraseña: ")
        if not password:
            print("   Error: La contraseña no puede estar vacía.")
            continue
        password_confirm = getpass("   Confirmar contraseña: ")
        if password != password_confirm:
            print("   Error: Las contraseñas no coinciden. Intenta de nuevo.")
            password = "" # Resetear para volver a pedir

    # --- 4. Validar campos obligatorios ---
    if not dni or not nombre or not phone:
        print("\nError: Los campos DNI, Nombre y Teléfono son obligatorios.")
        return

    # --- 5. Crear el usuario usando el Manager ---
    #    El método create_user se encarga de hashear la contraseña.
    try:
        print(f"\nIntentando crear usuario con DNI: {dni}...")
        # Usamos **extra_fields para pasar los campos adicionales
        user = Usuario.objects.create_user(
            dni=dni,
            password=password,
            # Campos adicionales que definiste en tu modelo Usuario
            nombre=nombre,
            email=email if email else None, # Guardar None si está vacío
            phone=phone,
            rol=rol,
            negocio=negocio_instance, # Asignar la instancia de Negocio encontrada
            # Otros campos de AbstractUser (is_staff, etc.) tomarán valores por defecto (False)
        )
        print("-" * 30)
        print(f"¡ÉXITO! Usuario creado:")
        print(f"  ID: {user.id}")
        print(f"  DNI: {user.dni}")
        print(f"  Nombre: {user.nombre}")
        print(f"  Negocio: {user.negocio.nombre}")
        print(f"  Rol: {user.rol}")
        print("-" * 30)

    except IntegrityError:
        # Esto suele ocurrir si el DNI ya existe (porque lo definiste como unique=True)
        print(f"\nError: Ya existe un usuario con el DNI '{dni}'. No se pudo crear.")
    except Exception as e:
        print(f"\nError inesperado al intentar crear el usuario: {e}")

# --- Punto de entrada del Script ---
if __name__ == "__main__":
    print("Iniciando script para agregar usuario...")
    setup_django_environment()
    add_user_interactively()
    print("\nScript finalizado.")

Iniciando script para agregar usuario...
Entorno Django configurado exitosamente.

--- Agregar Nuevo Usuario ---

Negocios disponibles:
Ocurrió un error inesperado buscando el negocio: You cannot call this from an async context - use a thread or sync_to_async.

Script finalizado.
